# All Pairs Shortest Paths (Floyd-Warshall Algorithm)

### Shortest path in weighted graphs
> 2 types of shortest path problems of iterest

* Single source shortest path
  - Find the shortest path form a fixed vertex to every other vertex
  - Transport finished product from the factory (single source) to all the retail outlets
  - Courier company delivers items from distribution center (single source) to the addresses
  - Dijkstra's algorithm (non-negative weights), Bellman-Ford algorithm (allows negative weights)

* All pairs shortest paths
  - Find the shortest paths between every pair of vertices $i$ and $j$
  - Optimal airline, railway, road routes between cities
  - Run Dijkstra or Bellman-Ford from each vertex
  - Is there another way?

### Transitive Closure

* Adjacency matrix $A$ represents paths of length $1$
* Matrix multiplication, $A^2 = A \times A$
  - $A^2[i, j] = 1$ if there is a path of length 2 form $i$ to $j$
  - For some $k, A[i, k] = A[k, j] = 1$
* In general, $A^{l + 1} = A^l \times A$
  - $A^{l + 1}[i, j] = 1$ if there is a path of length $l + 1$ from $i$ to $j$
  - For some $k, A^l[i, j] = 1, A[k, j] = 1$
* $A^+ = A + A^2 + ... + A^{n - 1}$

**An alternative approach**
* $B^k[i, j] = 1$ is there is a path from $i$ to $j$ via the vertices $\{0, 1, ..., k - 1\}$
  - Constraint applies only to intermediate vertices between $i$ and $j$
  - $B^0[i, j] = 1$ if there is a direct edge
  - $B^0 = A$
* $B^{k + 1}[i, j] = 1$ if
  - $B^k[i, j] = 1$ - can already reach $j$ from $i$ via $\{0, 1, ..., k - 1\}$
  - $B^k[i, k] = 1$ and $B^k[k, j] = 1$ - use $\{0, 1, ..., k - 1\}$ to go from $i$ to $k$ and then from $k$ to $j$

### Warshall's Algorithm

* $B^k[i, j] = 1$ if there is a path from $i$ to $j$ via the vertices $\{0, 1, ..., k - 1\}$
* $B^0[i, j] = A[i, j]$
  - Direct edges, no intermediate vertices
* $B^{k + 1}[i, j] = 1$ if
  - $B^k[i, j] = 1$ or
  - $B^k[i, k] = 1$ and $B^k[k, j] = 1$
* The algorithm also computes transitive closure - Warshall's algorithm
* $B^n[i, j] = 1$ if there is some path from $i$ to $j$ with intermediate vertices in $\{0, 1, ..., n - 1\}$
* $B^n = A^+$
* We adapt the Warshall's algorithm to compute all-pairs shortest paths

### Floyd-Warshall Algorithm

* Let $SP^k[i, j]$ be the length of the shortest path from $i$ to $j$ via the vertices $\{0, 1, ..., k - 1\}$
* $SP^0[i, j] = W[i, j]$
  - No intermediate vertices, shortest path is weight of the direct edge
  - Assume $W[i, j] = \infty$ if $(i, j) \notin E$
* $SP^{k + 1}[i, j]$ is the minimum of
  - $SP^k[i, j]$
  - Shortest path using only $\{0, 1, ..., k - 1\}$
  - $SP^k[i, k] + SP^k[k, j]$
  - Combine the shortest path from $i$ to $k$ and $k$ to $j$
* $SP^n[i, j] = 1$ is the length of the shortest path overall from $i$ to $j$
  - Intermediate vertices lie in $\{0, 1, ..., n - 1\}$

### Implementation

* Shortest path matrix $SP$ is $n \times n \times (n + 1)$
* Initialize $SP[i, j, 0]$ to edge weight $W(i, j)$ or $\infty$ if not edge
* Update $SP[i, j, k]$ from $SP[i, j, k - 1]$ using the Floyd-Warshall update rule
* Time complexity is $O(n^3)$
* We only need $SP[i, j, k - 1]$ to compute $SP[i, j, k]$
* Maintain two "slices" $SP[i, j]$ $SP'[i, j]$, compute $SP'$ from $SP$, copy $SP'$ to $SP$, save space

In [ ]:
def floyd_warshall(WMat):
  (rows, cols, x) = WMat.shape
  infinity = np.max(WMat) * rows * rows + 1
  Sp = np.zeros(shape=(rows, cols, cols + 1))

  for i in range(rows):
    for j in range(cols):
      SP[i, j, 0] = infinity
  
  for i in range(rows):
    for j in range(cols):
      if WMat[i, j, 0] == 1:
        SP[i, j, 0] = WMat[i, j, 1]
  
  for k in range(1, cols + 1):
    for i in range(rows):
      for j in range(cols):
        SP[i, j, k] = min(SP[i, j, k - 1]), SP[i, k - 1, k - 1] + SP[k - 1, j, k - 1])
  
  return SP[:, :, cols]

### Summary

* Warshall's algorithm is an alternative way to compute transitive closure
  - $B^k[i, j] = 1$ if we can reach $j$ from $i$ using the vertices in $\{0, 1, ..., k - 1\}$
* Adapt Warshall's algorithm to compute all pairs of shortest paths
  - $SP^k[i, j]$ if the length of the shortest path from $i$ to $j$ using vertices in $\{0, 1, ..., k - 1\}$
  - $SP^n[i, j]$ is the length of the overall shortest path
* Works with negative edge weights assuming no negative cycles
* Simple nested loop implementation, time $O(n^3)$
* Space can be limited to $O(n^2)$ by re-using 2 "slices" $SP$ and $SP'$